# Decomposição LU

A idéia é criar 2 matrizes auxiliares:

$A = LU$

Assim, substituindo na equação $Ax = b$:

$LUx = b$

Fazendo:

$Ux = y$

temos:

$Ly = b$

Para resolver o sistema, primeiro o sistema $Ly = b$ achando o valor de $y$.

Depois, resolvemos o sistema $Ux = y$ e achamos $x$.

Qual a vantagem de utilizar a decomposição LU?

* Computacionalmente mais eficiente (quando aplicado para vários vetores $b$, usando a mesma matriz $A$)
* Cálculo da inversa de uma matriz

## Exemplo:

Resolva o seguinte sistema de equações:

\begin{aligned}
\begin{bmatrix}
1 & 1 & 1 \\
2 & 1 & -1 \\
2 & -1 & 1 
\end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = 
\begin{bmatrix} b_1 \\ b_2 \\ b_3 \end{bmatrix}
\end{aligned}

para:

$ b = \begin{bmatrix} -2 \\ 1 \\ 3 \end{bmatrix}$

e

$ b = \begin{bmatrix} 1 \\ 1 \\ 3 \end{bmatrix}$

In [75]:
import numpy as np
import scipy.linalg 

def lu(A):
    n = np.shape(A)[0]
    L = np.identity(n)

    for PIVO in range(n):
        for LIN in range(PIVO+1, n):
            C = A[LIN,PIVO]/A[PIVO,PIVO]
            L[LIN,PIVO] = C
            A[LIN,:] = A[LIN,:] - C*A[PIVO,:]
    return L,A
    
# https://johnfoster.pge.utexas.edu/numerical-methods-book/LinearAlgebra_LU.html
def forward_substitution(L, b):
    #Get number of rows
    n = L.shape[0]
    
    #Allocating space for the solution vector
    y = np.zeros_like(b, dtype=np.double);
    
    #Here we perform the forward-substitution.  
    #Initializing  with the first row.
    y[0] = b[0] / L[0, 0]
    
    #Looping over rows in reverse (from the bottom  up),
    #starting with the second to last row, because  the 
    #last row solve was completed in the last step.
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

def back_substitution(U, y):
    #Number of rows
    n = U.shape[0]
    
    #Allocating space for the solution vector
    x = np.zeros_like(y, dtype=np.double);

    #Here we perform the back-substitution.  
    #Initializing with the last row.
    x[-1] = y[-1] / U[-1, -1]
    
    #Looping over rows in reverse (from the bottom up), 
    #starting with the second to last row, because the 
    #last row solve was completed in the last step.
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x

A = np.array([[1,1,1], [2,1,-1], [2,-1,1]])
#A = np.array([[3, 2, 4], [1, 1, 2], [4, 3, -2]])

P,L,U = scipy.linalg.lu(A)

print('LU de A:\n')
L,U = lu(A)


print(P, '\n')
print(L, '\n')
print(U, '\n')
print(P @ L @ U)

LU de A:

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]] 

[[1. 0. 0.]
 [2. 1. 0.]
 [2. 3. 1.]] 

[[ 1  1  1]
 [ 0 -1 -3]
 [ 0  0  8]] 

[[ 2. -1.  1.]
 [ 1.  1.  1.]
 [ 2.  1. -1.]]


In [77]:
b = np.array([-2,1,3])
#b = np.array([1,1,3])

y = forward_substitution(L, P.T @  b)
y = forward_substitution(L, b)
x = back_substitution(U, y)

print(A)
print(np.linalg.solve(A,b))

[[ 1  1  1]
 [ 0 -1 -3]
 [ 0  0  8]]
[-0.25  -2.125  0.375]


In [57]:
LU, P = scipy.linalg.lu_factor(A)

b = np.array([-2,1,3])
x = scipy.linalg.lu_solve((LU,P), b)
print(x)

b = np.array([1,1,3])
x = scipy.linalg.lu_solve((LU,P), b)
print(x)

[ 1. -2. -1.]
[ 1.  -0.5  0.5]
